In [3]:
#RadioML(DeepSig) dataset file unloading and creation of Tensorflow Dataset

import h5py
import json
import tensorflow as tf
import numpy as np
from numpy import argwhere

# Open the dataset
hdf5_file = h5py.File("../dataset/2018.01/DeepSig_XYZ_OSC.0001_1024_Shuffled.hdf5",  'r')
# Load the modulation classes. You can also copy and paste the content of classes-fixed.txt.
modulation_classes = json.load(open("../dataset/2018.01/classes-fixed.json", 'r'))
#List groups of the hdf5 file
list(hdf5_file.keys())

# Read the HDF5 groups
# Dataset is to large for allocated RAM, so every fourth frame will be used, resulting in a fourth of the dataset being used
data = hdf5_file['X'][0:1277952:8]
modulation_onehot = hdf5_file['Y'][0:1277952:8]
snr = hdf5_file['Z'][0:1277952:8]

#creation of test dataset
test_data = data[1277952::128]
test_mod = modulation_onehot[1277952::128]
test_snr = snr[1277952::128]

# Closes the file
hdf5_file.close()

#converting from onehot to integer
mod_array = np.zeros(len(modulation_onehot))
i = 0
for x in modulation_onehot:
    mod_array[i] = np.where(x==1)[0][0]
    i = i + 1


    
#taking the three arrays and converting them to a tf dataset object
dataset = tf.data.Dataset.from_tensor_slices((data, mod_array))
test_dataset = tf.data.Dataset.from_tensor_slices((test_data, test_mod))

2021-11-01 14:38:54.504394: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-11-01 14:38:54.504421: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-11-01 14:38:54.504437: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (bolt): /proc/driver/nvidia/version does not exist
2021-11-01 14:38:54.505248: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-01 14:38:54.508980: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1308622848 exceeds 10%

In [4]:
#Tensorflow Dataset Object normalization, preprocessing, and customization block
batch_size = 1024
buf_size = 26624
drop_rem = True

dataset = dataset.shuffle(buffer_size=buf_size)
dataset = dataset.batch(batch_size, drop_remainder=drop_rem)

In [5]:
#model variables
img_width, img_height, img_channels = (1024, 2, 1)
opt = 'Adam'
lss = 'sparse_categorical_crossentropy'


model = tf.keras.models.Sequential([
      tf.keras.layers.Conv1D(64, 3, activation='relu', 
                  input_shape=(img_width, img_height)),
      tf.keras.layers.MaxPooling1D(2, 2),
      tf.keras.layers.Conv1D(64, 3, activation='relu'),
      tf.keras.layers.MaxPooling1D(2),
      tf.keras.layers.Conv1D(64, 3, activation='relu'),
      tf.keras.layers.MaxPooling1D(2),
      tf.keras.layers.Conv1D(64, 3, activation='relu'),
      tf.keras.layers.MaxPooling1D(2),
      tf.keras.layers.Conv1D(64, 3, activation='relu'),
      tf.keras.layers.MaxPooling1D(2),
      tf.keras.layers.Conv1D(64, 3, activation='relu'),
      tf.keras.layers.MaxPooling1D(2),
      tf.keras.layers.Conv1D(64, 3, activation='relu'),
      tf.keras.layers.MaxPooling1D(2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128, activation=tf.nn.selu),
      tf.keras.layers.Dense(128, activation=tf.nn.selu),
      tf.keras.layers.Dense(24, activation=tf.nn.softmax)
    ])

model.compile(optimizer=opt, loss=lss, metrics=['accuracy'])


In [6]:
model.fit(dataset, epochs=5)

2021-11-01 14:39:01.732893: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1308622848 exceeds 10% of free system memory.
2021-11-01 14:39:02.373636: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1308622848 exceeds 10% of free system memory.


Epoch 1/2


2021-11-01 14:39:03.326538: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


156/156 [==============================] - 119s 760ms/step - loss: 1.3514 - accuracy: 0.3795


2021-11-01 14:41:02.095988: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1308622848 exceeds 10% of free system memory.


Epoch 2/2
156/156 [==============================] - 117s 750ms/step - loss: 0.9088 - accuracy: 0.5505


In [7]:
# Open the dataset
hdf5_file = h5py.File("../dataset/2018.01/GOLD_XYZ_OSC.0001_1024.hdf5",  'r')

to_test = hdf5_file['X'][:1277953:16384]
to_test_label = hdf5_file['Y'][:1277953:16384]

#converting from onehot to integer
mod_array_test = np.zeros(len(to_test_label))
i = 0
for x in to_test_label:
    mod_array_test[i] = np.where(x==1)[0][0]
    i = i + 1

# Closes the file
hdf5_file.close()

#print(to_test)
print(mod_array_test)

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.
  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  3.  3.  3.  3.  3.  3.  3.  3.
  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  4.  4.  4.  4.
  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  5.
  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.
  5.  5.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.
  6.  6.  6.  6.  6.  6.  7.  7.  7.  7.  7.  7.  7.  7.  7.  7.  7.  7.
  7.  7.  7.  7.  7.  7.  7.  7.  7.  8.  8.  8.  8.  8.  8.  8.  8.  8.
  8.  8.  8.  8.  8.  8.  8.  8.  8.  8.  8.  8.  9.  9.  9.  9.  9.  9.
  9.  9.  9.  9.  9.  9.  9.  9.  9.  9.  9.  9.  9.  9.  9. 10. 10. 10.
 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10.
 10. 11. 11. 11. 11. 11. 11. 11. 11. 11. 11. 11. 11

In [8]:
print("Prediction on test data")
results = model.predict(to_test)
print("test loss, test acc:", results)

Prediction on test data
test loss, test acc: [[1.1106449e-01 9.9796414e-02 1.0953203e-01 ... 7.8018263e-07
  7.8129949e-07 7.3869137e-06]
 [2.0148498e-01 1.7246929e-01 1.8340640e-01 ... 4.6852418e-07
  6.6271281e-07 6.7601090e-06]
 [1.5505995e-01 1.2521376e-01 1.3724317e-01 ... 7.5116816e-07
  7.3983131e-07 7.0773672e-06]
 ...
 [5.2856556e-08 1.3074509e-06 4.7350909e-06 ... 2.7933654e-06
  1.1440151e-06 7.5735733e-07]
 [2.0582641e-08 7.7505695e-07 3.3765525e-06 ... 2.3690454e-06
  9.8721091e-07 4.9750838e-07]
 [1.4732208e-07 1.5944645e-06 5.2177770e-06 ... 3.6505903e-06
  1.5882695e-06 1.1020229e-06]]


In [11]:
model.save("../dataset/2018.01/savedModel")

2021-11-01 14:59:49.063108: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ../dataset/2018.01/savedModel/assets


In [9]:
predictions = [p.argmax() for p in results]

# print(len(results[0]))

correct_guesses = 0
for c, a in zip(mod_array_test, predictions):
    if c == a:
        correct_guesses += 1
        
accuracy = '{0:.6}'.format(correct_guesses/len(predictions))
        
print(f'{correct_guesses} / {len(predictions)}')
print(accuracy)

print(*[(c, a, c == a) for c, a in zip(mod_array_test, predictions)], sep="\n")

71 / 512
0.138672
(0.0, 5, False)
(0.0, 0, True)
(0.0, 3, False)
(0.0, 3, False)
(0.0, 5, False)
(0.0, 3, False)
(0.0, 0, True)
(0.0, 0, True)
(0.0, 0, True)
(0.0, 0, True)
(0.0, 0, True)
(0.0, 0, True)
(0.0, 0, True)
(0.0, 0, True)
(0.0, 0, True)
(0.0, 0, True)
(0.0, 0, True)
(0.0, 0, True)
(0.0, 0, True)
(0.0, 0, True)
(0.0, 0, True)
(0.0, 0, True)
(1.0, 5, False)
(1.0, 2, False)
(1.0, 5, False)
(1.0, 5, False)
(1.0, 3, False)
(1.0, 0, False)
(1.0, 1, True)
(1.0, 1, True)
(1.0, 0, False)
(1.0, 1, True)
(1.0, 1, True)
(1.0, 1, True)
(1.0, 1, True)
(1.0, 1, True)
(1.0, 2, False)
(1.0, 1, True)
(1.0, 1, True)
(1.0, 1, True)
(1.0, 2, False)
(1.0, 2, False)
(1.0, 1, True)
(2.0, 3, False)
(2.0, 5, False)
(2.0, 5, False)
(2.0, 3, False)
(2.0, 0, False)
(2.0, 0, False)
(2.0, 0, False)
(2.0, 2, True)
(2.0, 2, True)
(2.0, 1, False)
(2.0, 2, True)
(2.0, 2, True)
(2.0, 2, True)
(2.0, 2, True)
(2.0, 2, True)
(2.0, 1, False)
(2.0, 2, True)
(2.0, 1, False)
(2.0, 2, True)
(2.0, 1, False)
(2.0, 2, Tr